Reference: https://www.pytry3g.com/entry/gensim-wikipedia-word2vec
で、PathLineSentencesを使い、簡素化

#前準備

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/NLP/

/content/drive/My Drive/Colab Notebooks/NLP20211H
/content/drive/My Drive/Colab Notebooks/NLP20211H


In [ ]:
!pip install mecab-python3
!pip install unidic-lite

     |████████████████████████████████| 491kB 14.5MB/s 
     |████████████████████████████████| 47.4MB 65kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-cp37-none-any.whl size=47658825 sha256=28da976721ade94e85a7e7f918f53f4fd779d6ce528b7411329cd1025508a10f
  Stored in directory: /root/.cache/pip/wheels/20/48/8d/b66d8361a27f58f41ec86640e4fd2640de0403a6367511eab7
Successfully built unidic-lite


#Wikipediaダウンロード

In [ ]:
pip install wikiextractor

     |████████████████████████████████| 51kB 7.0MB/s 


#Mecab版

##テキスト抽出、分かち書き 

In [ ]:
!python -m wikiextractor.WikiExtractor jawiki-latest-pages-articles.xml.bz2 -b 10M -o articles

INFO: Finished 1-process extraction of 1213131 articles in 6278.3s (193.2 art/s)
INFO: total of page: 1760369, total of articl page: 1213131; total of used articl page: 1213131

2.88GB

In [ ]:
!pip install bs4
!pip install lxml
!python -m pip install -U gensim
!pip install tqdm

In [ ]:
import MeCab
import codecs
import glob
from bs4 import BeautifulSoup
from datetime import datetime

tagger = MeCab.Tagger('-Owakati')

def save(src):
    with codecs.open(src, 'r', 'utf-8') as f:
        soup = BeautifulSoup(f.read(), "lxml")
        doc_tags = soup.find_all("doc")
        sentences = []
        dst = "./contents/wiki{}".format(len(glob.glob("./contents/wiki*"))+1)
        for doc in doc_tags:
          if len(doc.text)>1000000:
            print('doc parse failed {}'.format(doc.text.splitlines()[:3]))
            break
          content = "".join(doc.text.splitlines()[3:])
          words = tagger.parse(content)
          if type(words) is not str:
            print("tagger failed {}".format(doc.text.splitlines()[:3]))
            break
          content = " ".join(tagger.parse(content).split())
          sentences.append(content)
        print(*sentences, sep="\n", file=codecs.open(dst, 'w', 'utf-8'))


# 1. globを使ってarticlesにある３つのフォルダのパスを取得。
dirlist = glob.glob("./articles/*")
# dirlist = ["articles/AA", "articles/AB", "articles/AC"]
for dirname in dirlist:
    # 2. AA, AB, ACの中の全てのファイルのパスを取得。
    for src in glob.glob(dirname+"/*"):
        # 3. ファイルをひとつずつ処理していく。
        # src = "articles/AA/wiki_00"
        print("{}:\t{}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), src))
        save(src)

##gensim skipgramモデル作成

In [ ]:
import glob
import logging
from gensim.models.word2vec import Word2Vec, PathLineSentences

SIZE = 128
MIN_COUNT = 500
WINDOW = 5
SG = 1
dirname = f"size{SIZE}-min_count{MIN_COUNT}-window{WINDOW}-sg{SG}"
if not "./model/"+dirname in glob.glob("./model/*"):
    os.mkdir("./model/"+dirname)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# モデルを作る
model = Word2Vec(
    PathLineSentences('./contents'), 
    vector_size=SIZE, min_count=MIN_COUNT, window=WINDOW, sg=SG, workers=4)
model.save("./model/"+dirname+'/wikipedia.model')

In [ ]:
dirname = "size{}-min_count{}-window{}-sg{}".format(SIZE, MIN_COUNT, WINDOW, SG)
model = Word2Vec.load("./model/"+dirname+"/wikipedia.model")
print(model.wv.most_similar('ロボット', topn=5))
print(model.wv.most_similar('コンピュータ', topn=5))

2021-05-10 08:23:36,574 : INFO : loading Word2Vec object from ./model/size128-min_count500-window5-sg1/wikipedia.model
2021-05-10 08:23:37,067 : INFO : loading wv recursively from ./model/size128-min_count500-window5-sg1/wikipedia.model.wv.* with mmap=None
2021-05-10 08:23:37,068 : INFO : setting ignored attribute vectors_norm to None
2021-05-10 08:23:37,073 : INFO : loading vocabulary recursively from ./model/size128-min_count500-window5-sg1/wikipedia.model.vocabulary.* with mmap=None
2021-05-10 08:23:37,074 : INFO : loading trainables recursively from ./model/size128-min_count500-window5-sg1/wikipedia.model.trainables.* with mmap=None
2021-05-10 08:23:37,079 : INFO : setting ignored attribute cum_table to None
2021-05-10 08:23:37,079 : INFO : loaded ./model/size128-min_count500-window5-sg1/wikipedia.model
2021-05-10 08:23:37,177 : INFO : precomputing L2-norms of word weight vectors


[('メカ', 0.7666413187980652), ('ロボ', 0.761128842830658), ('人型', 0.7293363213539124), ('アンドロイド', 0.7178972363471985), ('サイボーグ', 0.7154169082641602)]
[('コンピューター', 0.9090123176574707), ('パソコン', 0.775415301322937), ('ソフトウェア', 0.7750289440155029), ('ハードウェア', 0.7698166966438293), ('ワークステーション', 0.7591724395751953)]


#SentencePiece版

##テキスト抽出

In [ ]:
!pip install bs4
!pip install lxml
!python -m pip install -U gensim
!pip install tqdm

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [ ]:
import codecs
import glob
from bs4 import BeautifulSoup
from datetime import datetime

def save(src):
    with codecs.open(src, 'r', 'utf-8') as f:
        soup = BeautifulSoup(f.read(), "lxml")
        doc_tags = soup.find_all("doc")
        sentences = []
        dst = "./texts/wiki{}".format(len(glob.glob("./texts/wiki*"))+1)
        for doc in doc_tags:
          if len(doc.text)>1000000:
            print('doc parse failed {}'.format(doc.text.splitlines()[:3]))
            break
          content = ''.join(doc.text.splitlines()[3:])
          content = content.replace('。','。\n')
          sentences.append(content)
        print(*sentences, sep="\n", file=codecs.open(dst, 'w', 'utf-8'))

# 1. globを使ってarticlesにある３つのフォルダのパスを取得。
dirlist = glob.glob("./articles/*")
# dirlist = ["articles/AA", "articles/AB", "articles/AC"]
for dirname in dirlist:
    # 2. AA, AB, ACの中の全てのファイルのパスを取得。
    for src in glob.glob(dirname+"/*"):
        # 3. ファイルをひとつずつ処理していく。
        # src = "articles/AA/wiki_00"
        print("{}:\t{}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), src))
        save(src)

##SentencePieceモデル作成

In [ ]:
# Reference https://www.pytry3g.com/entry/how-to-use-sentencepiece
# Reference https://qiita.com/taku910/items/7e52f1e58d0ea6e7859c
!pip install sentencepiece
import sentencepiece as spm

In [ ]:
import codecs
import glob

def iter_files():
  for textfile in glob.glob("./texts/*"):
    with codecs.open(textfile, 'r', 'utf-8') as f:
      lines = f.read().splitlines()
      for idx, line in enumerate(lines):
        if idx%5==0 and len(line)>30: # spm consume ram too much, so skip some
          #print(line)
          yield line

#for l in iter_files(): print(l)
spm.SentencePieceTrainer.Train(sentence_iterator=iter_files(),  model_prefix='sentencepiece', character_coverage=0.9995, vocab_size=30000, model_type='unigram')


./texts/wiki319
./texts/wiki320


##分かち書き

In [ ]:
#trial
sp = spm.SentencePieceProcessor()
sp.Load("sentencepiece.model")
print(sp.EncodeAsPieces('横浜工場で、情報処理装置を開発する'))
print(sp.EncodeAsPieces('ペットの犬がしっぽを振りながら迎えに駆け寄ってきた。'))

['▁', '横浜', '工場で', '、', '情報処理', '装置', 'を開発する']
['▁', 'ペット', 'の', '犬', 'が', 'し', 'っぽ', 'を振り', 'ながら', '迎え', 'に', '駆け', '寄', 'ってきた', '。']


In [ ]:
import codecs
import glob
from bs4 import BeautifulSoup
from datetime import datetime

def save(src):
    with codecs.open(src, 'r', 'utf-8') as f:
        sentences = []
        dst = "./contents/wiki{}".format(len(glob.glob("./contents/wiki*"))+1)
        lines = f.read().splitlines()
        for line in lines:
          words = " ".join(sp.EncodeAsPieces(line))
          sentences.append(words)
        print(*sentences, sep="\n", file=codecs.open(dst, 'w', 'utf-8'))

sp = spm.SentencePieceProcessor()
sp.Load("sentencepiece.model")
for src in glob.glob("./texts/*"):
    # 3. ファイルをひとつずつ処理していく。
    # src = "articles/AA/wiki_00"
    print("{}:\t{}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), src))
    save(src)

##gensim skipgramモデル作成

In [ ]:
import glob
import logging
from gensim.models.word2vec import Word2Vec, PathLineSentences
import os

SIZE = 100
MIN_COUNT = 300
WINDOW = 5
SG = 1
dirname = f"size{SIZE}-min_count{MIN_COUNT}-window{WINDOW}-sg{SG}"
if not "./model/"+dirname in glob.glob("./model/*"):
    os.mkdir("./model/"+dirname)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# モデルを作る
model = Word2Vec(
    PathLineSentences('./contents'), 
    vector_size=SIZE, min_count=MIN_COUNT, window=WINDOW, sg=SG, workers=4)
model.save("./model/"+dirname+'/wikipedia.model')

In [ ]:
dirname = "size{}-min_count{}-window{}-sg{}".format(SIZE, MIN_COUNT, WINDOW, SG)
model = Word2Vec.load("./model/"+dirname+"/wikipedia.model")
print(model.wv.most_similar('ロボット', topn=5))
print(model.wv.most_similar('コンピュータ', topn=5))
print(model.wv.most_similar('電話', topn=5))

2021-05-13 11:27:57,045 : INFO : loading Word2Vec object from ./model/size100-min_count300-window5-sg1/wikipedia.model
2021-05-13 11:27:57,110 : INFO : loading wv recursively from ./model/size100-min_count300-window5-sg1/wikipedia.model.wv.* with mmap=None
2021-05-13 11:27:57,115 : INFO : setting ignored attribute cum_table to None
2021-05-13 11:27:57,478 : INFO : Word2Vec lifecycle event {'fname': './model/size100-min_count300-window5-sg1/wikipedia.model', 'datetime': '2021-05-13T11:27:57.478698', 'gensim': '4.0.1', 'python': '3.7.10 (default, May  3 2021, 02:48:31) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.109+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


[('メカ', 0.8104596138000488), ('ロボ', 0.7993384003639221), ('トランスフォーマー', 0.7672944068908691), ('ゾイド', 0.7414247989654541), ('モビルスーツ', 0.7383398413658142)]
[('コンピューター', 0.9479199051856995), ('パーソナルコンピュータ', 0.8332098722457886), ('ソフトウェア', 0.8276858329772949), ('グラフィックス', 0.8234474658966064), ('ハードウェア', 0.8160702586174011)]
[('通話', 0.7724155783653259), ('電話番号', 0.765082061290741), ('回線', 0.7504477500915527), ('携帯電話', 0.7383388876914978), ('無線', 0.7051928639411926)]
